## Gerekli kütüphaneler:

In [1]:
import sys

sys.path.append("../..")

import os
import cv2
import numpy as np
import joblib
from tqdm import tqdm

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split


from utils.ml_features.preprocesing import extract_features, apply_pca
from utils.ml_features.grid_search import grid_search

## Veri yolu ve etiketler:

In [2]:
data_dir = "../../data/skyview"
class_names = sorted(os.listdir(data_dir))

## Feature çıkarma:

In [3]:
x_data, y_data = [], []

for label, cls in enumerate(class_names):
    cls_path = os.path.join(data_dir, cls)
    print("İşlenen class: ", cls)
    for fname in tqdm(os.listdir(cls_path), desc=f"{cls}", leave=False):
        img_path = os.path.join(cls_path, fname)
        img = cv2.imread(img_path)
        if img is None:
            continue
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (128, 128))
        features = extract_features(img)
        x_data.append(features)
        y_data.append(label)

x_data = np.array(x_data)
y_data = np.array(y_data)
print("Toplam örnek: ", len(x_data), " | Feature boyutu: ",  x_data.shape[1])

İşlenen class:  Agriculture


İşlenen class:  Airport


İşlenen class:  Beach


İşlenen class:  City


İşlenen class:  Desert


...

İşlenen class:  Port


İşlenen class:  Railway


İşlenen class:  Residential


İşlenen class:  River


Toplam örnek:  12000  | Feature boyutu:  1803


## PCA kullanarak boyut azaltımı:

In [4]:
x_reduced = apply_pca(x_data, n_components=100)

## Veriyi bölme:

In [5]:
x_train, x_test, y_train, y_test = train_test_split(x_reduced, y_data, test_size=0.2, stratify=y_data, random_state=42)

## Grid search ile model eğitme:

In [ ]:
models = {
    "random_forest": (
        RandomForestClassifier(), {
            "n_estimators": [75, 100], 
            "max_depth": [10, 20]
        }
    ),

    "gradient_boosting": (
        GradientBoostingClassifier(), {"n_estimators": [100, 200]}
    ),

    "multilayer_perceptron": (
        MLPClassifier(max_iter=300),
        {
            "hidden_layer_sizes": [(100,), (100, 50), (128, 64)],
            "alpha": [0.0001, 0.001],
            "learning_rate_init": [0.001, 0.01]
        }
    )
}

trained_models = {}

print("\nModellerin eğitimi başladı.\n")

for name, (model, params) in models.items():
    print(name.upper(), " eğitimi başladı.")
    best_model, best_score, best_params = grid_search(model, params, x_train, y_train, cv=5)
    trained_models[name] = best_model
    joblib.dump(best_model, "../../models/ml_features/skyview_" + name + ".pkl")
    print("\n", name.upper(), " için en iyi skor: ", best_score, " | Parametreler: ", best_params + "\n")


Modellerin eğitilimi başladı.

RANDOM_FOREST  eğitimi başladı.
Denenecek hiperparametre kombinasyon sayısı:  2


RandomForestClassifier Grid Search:  50%|█████     | 1/2 [00:25<00:25, 25.47s/it]

Parametreler:  {'n_estimators': 100}  | Skor:  0.7291666666666666  | Süre:  25.466294527053833 sn


RandomForestClassifier Grid Search: 100%|██████████| 2/2 [01:10<00:00, 35.16s/it]

Parametreler:  {'n_estimators': 200}  | Skor:  0.7396874999999999  | Süre:  44.84531283378601 sn



 RANDOM_FOREST  için en iyi skor:  0.7396874999999999  | Parametreler:  {'n_estimators': 200}
GRADIENT_BOOSTING  eğitimi başladı.
Denenecek hiperparametre kombinasyon sayısı:  2


GradientBoostingClassifier Grid Search:   0%|          | 0/2 [14:09<?, ?it/s]


TypeError: unsupported operand type(s) for +: 'int' and 'str'